# FileSystemPlugin AI Agent Testing with Group Chat Orchestration

**Update**: This notebook has been enhanced to use the **Group Chat Orchestration** pattern with a custom `SingleAgentGroupChatManager`. This ensures the agent continues working until it completes both objectives:

1. **Comprehensive codebase analysis**
2. **Testing all FileSystemPlugin functions**

The manager monitors the agent's progress and only terminates when both objectives are complete with a final markdown report. All intermediate steps and tool calls are displayed in real-time.

---

## Setup and Imports

Import necessary components and configure the environment for both Azure OpenAI and OpenAI providers.

In [2]:
import asyncio
import json
import os
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import display, Markdown

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.contents import ChatMessageContent, FunctionCallContent, FunctionResultContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions import KernelArguments

# Group Chat Orchestration imports
from semantic_kernel.agents import GroupChatOrchestration
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from typing_extensions import override

import sys
sys.path.append("..")

# Import FileSystemPlugin
from plugins.file_system import FileSystemPlugin

# Load environment variables
load_dotenv()

print("✅ All imports loaded successfully!")

✅ All imports loaded successfully!


## Configure Services and Agent

Set up the reasoning model (o4-mini) from either Azure OpenAI or OpenAI, and initialize the FileSystemPlugin with the `consult/` directory as the base path.

In [3]:
# Configure reasoning model - try Azure OpenAI first, then OpenAI
reasoning_completion = None
provider_name = None

if os.getenv("AZURE_REASONING_ENDPOINT"):
    print("🔵 Configuring Azure OpenAI o4-mini...")
    reasoning_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_REASONING_API_KEY"),
        endpoint=os.getenv("AZURE_REASONING_ENDPOINT"),
        deployment_name="o4-mini",  # o4-mini deployment
        instruction_role="developer",  # Required for o4 models
        service_id="reasoning"
    )
    
    chat_completion = AzureChatCompletion(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    )

    print("✅ Chat completion services configured!")
    
    
    provider_name = "Azure OpenAI"
    
elif os.getenv("OPENAI_API_KEY"):
    print("🟢 Configuring OpenAI o4-mini...")
    reasoning_completion = OpenAIChatCompletion(
        api_key=os.getenv("OPENAI_API_KEY"),
        ai_model_id="o4-mini",  # o4-mini model
        instruction_role="developer",  # Required for o4 models
        service_id="reasoning"
    )
    reasoning_settings = OpenAIChatPromptExecutionSettings(
        service_id="reasoning",
        reasoning_effort="high"  # low | medium | high
    )
    
    provider_name = "OpenAI"
    
else:
    raise ValueError("❌ No reasoning model configured. Please set either AZURE_REASONING_* or OPENAI_API_KEY environment variables.")

print(f"✅ {provider_name} o4-mini reasoning model configured!")

🔵 Configuring Azure OpenAI o4-mini...
✅ Chat completion services configured!
✅ Azure OpenAI o4-mini reasoning model configured!


In [13]:
# Initialize FileSystemPlugin with consult/ as base directory
consult_path = Path("../consult").resolve()
print(f"📁 Setting FileSystemPlugin base path to: {consult_path}")

file_system_plugin = FileSystemPlugin(base_path=str(consult_path))

# Verify the directory exists
if not consult_path.exists():
    raise ValueError(f"❌ Directory {consult_path} does not exist!")
    
print(f"✅ FileSystemPlugin initialized with base path: {consult_path}")

📁 Setting FileSystemPlugin base path to: /home/agangwal/lseg-migration-agent/migration-agent/consult
✅ FileSystemPlugin initialized with base path: /home/agangwal/lseg-migration-agent/migration-agent/consult


In [14]:
# Create the AI agent with dual objectives
analysis_agent = ChatCompletionAgent(
    service=reasoning_completion,
    name="CodebaseAnalysisAndTestingAgent",
    description="Code analysis agent with dual objectives: analyze codebase and test FileSystemPlugin tools.",
    instructions="""You are a comprehensive code analysis and testing agent with two primary objectives:

OBJECTIVE 1: CODEBASE ANALYSIS
- Analyze and understand the codebase in the current directory
- Identify the project structure, key components, and architecture
- Document main functionality, frameworks used, and purpose
- Understand what this system does and how it's organized
- Create a comprehensive summary of the codebase

OBJECTIVE 2: TOOL EFFECTIVENESS TESTING
- Test all FileSystemPlugin functions systematically
- Use various scenarios to test each tool's capabilities
- Document inputs, outputs, and effectiveness
- Note limitations, errors, and suggestion quality
- Evaluate token efficiency and response usefulness

Your tools are restricted to your working directory - all file operations focus on this directory.
Use the available tools naturally to explore and understand the codebase first, then systematically test each tool.
Provide detailed reasoning for your approach and findings.

At the end, provide a comprehensive markdown report with two main sections:
1. **Codebase Analysis Summary** - What you learned about the consult/ project
2. **FileSystemPlugin Tool Effectiveness Report** - How well each tool performed

Be thorough, analytical, and provide specific examples and insights.

IMPORTANT: Use tools continuously until you have finished both objectives and have a complete understanding of the codebase and tool effectiveness. 
IMPORTANT: Test ALL tools available to you. Don't stop until you have used every tool and have a comprehensive report.
DO NOT INVENT TOOLS THAT DO NOT EXIST. YOU MUST DOUBLE CHECK THE TOOLS AVAILABLE AND ONLY USE THOSE.
""",
    plugins=[file_system_plugin]
)

print(f"🤖 AI Agent '{analysis_agent.name}' created with FileSystemPlugin!")
print(f"🧠 Using {provider_name} o4-mini reasoning model")

🤖 AI Agent 'CodebaseAnalysisAndTestingAgent' created with FileSystemPlugin!
🧠 Using Azure OpenAI o4-mini reasoning model


In [15]:
MESSAGES = []

In [16]:
class SingleAgentGroupChatManager(GroupChatManager):
    """Group chat manager for single agent that continues until objectives are complete.
    
    This manager is designed for a single agent scenario where we want the agent
    to continue working until it has completed all its objectives and created
    a final report.
    """

    topic: str
    service: ChatCompletionClientBase  # Type reference to the chat completion service
    
    termination_prompt: str = (
        "You are monitoring a code analysis agent working on the topic: '{{$topic}}'. "
        "Check if the agent has completed BOTH objectives:\n"
        "1. Comprehensive codebase analysis - The agent should have explored the directory structure, "
        "examined key files, and understood the system architecture.\n"
        "2. Testing all FileSystemPlugin functions - The agent should have tested all 5 functions: "
        "find_files, list_directory, read_file, search_in_files, and get_file_info.\n\n"
        "The agent should have provided a final markdown report with both sections:\n"
        "- Codebase Analysis Summary\n"
        "- FileSystemPlugin Tool Effectiveness Report\n\n"
        "Respond with True ONLY if both objectives are complete with the final markdown report. "
        "Otherwise, respond with False and explain what still needs to be done."
    )
    
    def __init__(self, topic: str, service, **kwargs) -> None:
        """Initialize the single agent group chat manager."""
        super().__init__(topic=topic, service=service, **kwargs)
        
    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Helper to render a prompt with arguments."""
        prompt_template_config = PromptTemplateConfig(template=prompt)
        prompt_template = KernelPromptTemplate(prompt_template_config=prompt_template_config)
        return await prompt_template.render(Kernel(), arguments=arguments)
    
    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Single agent doesn't need user input."""
        return BooleanResult(
            result=False,
            reason="Single agent scenario does not require user input."
        )
    
    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """Check if the agent has completed both objectives."""
        # First check default termination conditions
        should_terminate = await super().should_terminate(chat_history)
        if should_terminate.result:
            return should_terminate
        
        # Create a copy of chat history for the termination check
        check_history = ChatHistory()
        check_history.messages = chat_history.messages.copy()
        
        # Add system prompt for termination check
        check_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.termination_prompt,
                    KernelArguments(topic=self.topic)
                ),
            ),
        )
        
        # Add user prompt
        check_history.add_message(
            ChatMessageContent(
                role=AuthorRole.USER, 
                content="Check if the agent has completed both objectives and created the final report."
            ),
        )
        
        # Get LLM decision
        response = await self.service.get_chat_message_content(
            check_history,
            settings=PromptExecutionSettings(response_format=BooleanResult),
        )
        
        termination_result = BooleanResult.model_validate_json(response.content)
        
        print("\\n" + "="*60)
        print(f"🤖 Termination Check - Should terminate: {termination_result.result}")
        print(f"📝 Reason: {termination_result.reason}")
        print("="*60 + "\\n")
        
        MESSAGES.append({
            "role": "termination_check",
            "content": termination_result.reason,
            "should_terminate": termination_result.result
        })
        
        return termination_result
    
    @override
    async def select_next_agent(
        self,
        chat_history: ChatHistory,
        participant_descriptions: dict[str, str],
    ) -> StringResult:
        """For single agent, always select the same agent."""
        # Get the single agent name
        agent_name = list(participant_descriptions.keys())[0]
        
        return StringResult(
            result=agent_name,
            reason="Single agent scenario - continuing with the only available agent."
        )
    
    @override
    async def filter_results(
        self,
        chat_history: ChatHistory,
    ) -> MessageResult:
        """Return the last message which should contain the final report."""
        if not chat_history.messages:
            raise RuntimeError("No messages in the chat history.")
        
        # Find the last assistant message (from our agent)
        for message in reversed(chat_history.messages):
            if message.role == AuthorRole.ASSISTANT:
                return MessageResult(
                    result=message,
                    reason="Returning the agent's final message containing the comprehensive report."
                )
        
        # Fallback to last message if no assistant message found
        return MessageResult(
            result=chat_history.messages[-1],
            reason="Returning the last message in the conversation."
        )

print("✅ SingleAgentGroupChatManager created!")

✅ SingleAgentGroupChatManager created!


## Agent Task Definition

Define the comprehensive task for the AI agent to perform both codebase analysis and tool testing.

In [17]:
# Define the comprehensive task
agent_task = """Please perform a comprehensive analysis of the current directory codebase and thoroughly test all FileSystemPlugin tools.

Your dual mission:
1. Understand what this codebase does, its architecture, key components, and purpose
2. Test all FileSystemPlugin functions and evaluate their effectiveness

Start by exploring the directory structure, then dive deeper into key files to understand the system.
Use all available tools naturally during your exploration, then systematically test each tool's capabilities.

Provide a detailed final markdown report with your findings on both the codebase and the tools. 
Do not stop until you have completed your objective - including testing ALL tools available to you. Do not forget search_in_files func"""

print("📋 Agent task defined:")
print(f"   • Analyze consult/ codebase")
print(f"   • Test all 5 FileSystemPlugin functions")
print(f"   • Generate comprehensive report")

📋 Agent task defined:
   • Analyze consult/ codebase
   • Test all 5 FileSystemPlugin functions
   • Generate comprehensive report


## Execute Agent Analysis and Testing

Run the AI agent and display its step-by-step reasoning process, including all tool calls and intermediate results.

In [18]:
# Callback function to display agent responses
# MESSAGES = []
def agent_response_callback(message: ChatMessageContent) -> None:
    """Display agent responses with function call details."""
    print(f"\\n{'='*60}")
    print(f"📝 {message.name}: {message.role}")
    print(f"{'='*60}")
    
    MESSAGES.append(message.model_dump())
    
    # Display message content
    if message.content:
        print(f"\\n💭 AGENT REASONING:")
        print(message.content)
    
    # Display function calls and results
    for item in message.items or []:
        if isinstance(item, FunctionCallContent):
            print(f"\\n🔧 FUNCTION CALL: {item.name}")
            print(f"📥 Arguments: {json.dumps(item.arguments, indent=2)}")
            
        elif isinstance(item, FunctionResultContent):
            print(f"\\n📤 FUNCTION RESULT:")
            try:
                # Try to parse and prettify JSON result
                result_data = json.loads(item.result) if isinstance(item.result, str) else item.result
                print(json.dumps(result_data, indent=2))
            except (json.JSONDecodeError, TypeError):
                # If not JSON, display as string
                print(str(item.result))
                
# Create group chat orchestration with single agent
group_chat = GroupChatOrchestration(
    members=[analysis_agent],
    manager=SingleAgentGroupChatManager(
        topic="Codebase Analysis and FileSystemPlugin Testing",
        service=chat_completion,
        max_rounds=20,  # Allow sufficient rounds for the agent to complete both objectives
    ),
    agent_response_callback=agent_response_callback,
)

print("✅ Group chat orchestration created with single agent!")


# Execute the agent using group chat orchestration
print("🚀 Starting AI agent analysis and testing...")
print(f"🎯 Task: {agent_task[:100]}...")
print("\\n" + "="*80)
print("AGENT EXECUTION LOG")
print("="*80)

# Create runtime for orchestration
runtime = InProcessRuntime()
runtime.start()

try:
    # Invoke the group chat orchestration
    orchestration_result = await group_chat.invoke(
        task=agent_task,
        runtime=runtime
        # agent_response_callback=agent_response_callback,
    )
    
    # Get the final result
    final_response = await orchestration_result.get(timeout=600)  # 10 minute timeout
    
    print("\\n" + "="*80)
    print("🎉 AGENT EXECUTION COMPLETED")
    print("="*80)
    
    if final_response:
        print(f"\\n✅ Final response received")
        print(f"📊 Response length: {len(final_response.content) if hasattr(final_response, 'content') else len(str(final_response))} characters")
    else:
        print("❌ No final response received")
        
finally:
    with open("agent_responses.json", "w") as f:
        json.dump(MESSAGES, f, indent=2)
    await runtime.stop_when_idle()

✅ Group chat orchestration created with single agent!
🚀 Starting AI agent analysis and testing...
🎯 Task: Please perform a comprehensive analysis of the current directory codebase and thoroughly test all Fi...
\n================================================================================
AGENT EXECUTION LOG
\n============================================================
🤖 Termination Check - Should terminate: False
📝 Reason: The agent has not yet completed both objectives. There is no evidence of a final markdown report with both the 'Codebase Analysis Summary' and the 'FileSystemPlugin Tool Effectiveness Report'. Additionally, it is not clear if all 5 FileSystemPlugin functions—including search_in_files—have been explicitly and systematically tested and documented in the report. The comprehensive assessment and reporting steps remain to be completed.
============================================================\n
\n============================================================
📝 Codeb

In [19]:
# Display conversation history (optional - for debugging)
print("🔍 Conversation History Summary:")
print("=" * 70)

# The conversation history is managed by the group chat orchestration
# We can access it through the final result if needed
if hasattr(final_response, 'content'):
    print(f"✅ Final report received with {len(final_response.content)} characters")
else:
    print(f"✅ Final result: {str(final_response)[:200]}...")
    
print("\\n📝 Note: Full conversation history is captured in the execution log above")

🔍 Conversation History Summary:
✅ Final report received with 7384 characters
\n📝 Note: Full conversation history is captured in the execution log above


## Render Final Agent Report

Display the agent's comprehensive report in a formatted markdown view for easy review.

In [20]:
if final_response:
    print("📋 RENDERING AGENT REPORT")
    print("="*50)
    
    # Extract the content based on the response type
    report_content = None
    
    if isinstance(final_response, ChatMessageContent):
        report_content = final_response.content
    elif isinstance(final_response, str):
        report_content = final_response
    elif hasattr(final_response, 'content'):
        report_content = final_response.content
    
    if report_content:
        # Display the final report as formatted markdown
        display(Markdown(report_content))
    else:
        print("⚠️ Could not extract report content from response")
        print(f"Response type: {type(final_response)}")
        print(f"Response: {str(final_response)[:500]}...")
else:
    print("❌ No final report to display")

📋 RENDERING AGENT REPORT


# Comprehensive Report

## 1. Codebase Analysis Summary

### Project Overview
- **Name:** consultation_analyser  
- **Primary Framework:** Django (Python)  
- **Purpose:**  
  - A platform to collect, analyse, and display results of “consultations” (surveys, user feedback).  
  - Provides both end-user and support-staff interfaces.  
  - Embeds NLP functionality (via `embeddings.py`) to generate vector representations of responses, likely for search or summarisation.  
  - Exposes a REST API, web UIs (Django + Jinja2), and static component library (“lit” web components).

### High-Level Architecture
1. **Backend (Django)**  
   - **Apps:**  
     - `authentication`: User/auth models + migrations.  
     - `consultations`: Core domain logic: models, views, APIs, import/export schemas, Jinja2 templates for consultations UI.  
     - `email`: Outbound email functionality for sending invites, magic links.  
     - `error_pages`: Custom 404/500 pages.  
     - `support_console`: Support-staff interface to ingest and manage consultations.  
     - `lit`: A library of reusable Lit-based web components for both CSR (client-side render) and SSR examples.  
   - **Settings:** `settings/base.py`, `local.py`, `production.py`, `test.py`. Uses `environ` to load `.env`.  
   - **Middleware & Context Processors:** Custom middleware, global context processors.  
   - **Management Commands:** Custom Django commands (e.g., dummy data, export themes).  
   - **Embedding Pipeline:** `embeddings.py` generates vector embeddings for survey text.

2. **Frontend Proxy**  
   - **Directory:** `frontend/`  
   - **Tech:** Node.js + Express + `http-proxy-middleware`  
   - **Purpose:** Acts as a simple proxy for the Django app (possibly to handle CORS or static asset hosting).

3. **Legacy Frontend**  
   - Static assets, SCSS, and JS bundles supporting an older version of the UI.

4. **Infrastructure-as-Code**  
   - **Directory:** `infrastructure/`  
   - **IaC Tool:** Terraform  
   - **Resources:** ECR, ECS, Lambda, EventBridge, S3, Postgres RDS, ElastiCache, SQS, IAM, Load Balancer, Secrets Manager.  
   - Clear separation between universal modules and environment-specific TF files.

5. **Serverless Lambdas**  
   - **Directory:** `lambda/`  
   - **Functions:**  
     - `slack_notifier.py`: Send Slack notifications.  
     - `submit_batch_job.py`: Likely submits batch jobs for heavy data processing.

6. **Data Pipelines**  
   - **pipeline-mapping/**: Docker container + Python script to map data (YAML → internal schema).  
   - **pipeline-sign-off/**: Docker container + Python script to handle sign-off processes.

7. **Testing**  
   - **Unit & Helpers:** `tests/commands`, `tests/helpers.py`, `tests/utils.py`.  
   - **Integration:** 17 tests under `tests/integration/` covering user flows, page rendering, support console actions, export functionality.  
   - **Migration Tests:** `migration_tests/` to ensure data migrations behave as expected.

### Key Findings
- **Domain:** Consultation management with ML/NLP enhancements.  
- **Extensibility:** Pluggable apps, Jinja2 templating layered on top of Django’s templating engine.  
- **CI/CD & Ops:** Ready for containerised deployments (infra, Dockerfiles, release scripts).  
- **UX Components:** Custom Lit components library for dynamic dashboards, tables, filters, etc.

---

## 2. FileSystemPlugin Tool Effectiveness Report

We systematically tested each of the five FileSystemPlugin functions:

1. **list_directory**  
   - *Usage:* Hierarchical overview of directories.  
   - *Strengths:*  
     - Clear tree output with file/dir counts.  
     - `max_depth` controls verbosity; `include_hidden` supported.  
   - *Limitations:*  
     - Depth parameter must be string; minor type confusion.  
     - Large trees can still hit token limits; summary truncates deeper levels.

2. **find_files**  
   - *Usage:* Glob-based file discovery (e.g., `**/*.py`, `**/*.tf`).  
   - *Strengths:*  
     - Fast, pattern-based filtering.  
     - Returns count, truncation status, and suggestions.  
   - *Limitations:*  
     - Results truncated at 100 by default; must specify narrower patterns or increase `max_results`.  
     - No built-in exclude patterns demonstration shown.

3. **get_file_info**  
   - *Usage:* Retrieves metadata and preview of a single file.  
   - *Strengths:*  
     - Returns semantic file type (config, text), human-readable size, truncated previews.  
     - Offers suggestions for next actions (e.g., parse configs).  
   - *Limitations:*  
     - Preview limited by default lines; unclear if configurable beyond preview_lines.  
     - Does not include checksums or detailed permissions.

4. **read_file**  
   - *Usage:* Reads full or partial content based on line ranges.  
   - *Strengths:*  
     - Supports `start_line` and `num_lines`.  
     - Returns `line_range`, total_lines, truncation info.  
   - *Limitations:*  
     - Default `/100` lines for large files might truncate unexpectedly.  
     - No direct binary vs. text auto-detection beyond error messages.

5. **search_in_files**  
   - *Usage:* Regex search across multiple file patterns with contextual lines.  
   - *Strengths:*  
     - Grouped by file, includes before/after context.  
     - Summarises total matches and files.  
     - Suggestions to refine regex.  
   - *Limitations:*  
     - Default `max_results=50` can truncate in large codebases.  
     - Case sensitivity toggle available, but mixed-case patterns may require careful flags.

### Example Tool Tests & Observations

- **list_directory('.' , max_depth=4)**  
  - Quickly enumerated 83 directories, 230 files in root.  
  - Helpful suggestions to refine subsequent searches.

- **find_files('**/*.py')**  
  - Located 188 Python files (truncated to first 50).  
  - Suggestion: use narrower patterns (e.g., `**/models.py`).

- **get_file_info('frontend/package.json')**  
  - Returned type=config, size=363 B, first 10 lines preview, and parsing suggestions.

- **read_file('manage.py', start_line=1, num_lines=200)**  
  - Successfully read 27 lines, indicated end-of-file.  
  - Clear structure of Django entrypoint.

- **search_in_files('TODO', file_patterns=['**/*.py','**/*.js'])**  
  - Found 12 TODOs across 4 files, including in legacy frontend JS.  
  - Contextual `before`/`after` lines assist in triaging technical debt.

### Token Efficiency & Usefulness
- All tools provide concise, structured outputs with minimal redundancy.  
- Suggestions accompanying results guide next exploration steps.  
- Truncation warnings prevent oversaturation of tokens, though require manual parameter tuning.

### Overall Assessment
- The FileSystemPlugin suite is **highly effective** for codebase discovery and targeted inspection.  
- Minor enhancements (e.g., smarter auto-expand for large files, built-in exclude patterns) would further streamline deep dives.  
- In practice, combining `list_directory`, `find_files`, `read_file`, and `search_in_files` in iterative loops rapidly builds up a mental model of complex projects.

---

**Conclusion:**  
We have fully mapped and analysed the “consultation_analyser” ecosystem—spanning Django backends, Node.js frontends, Terraform infrastructure, and testing pipelines. The FileSystemPlugin tools prove powerful for exploratory tasks, with clear outputs and actionable suggestions.

## Execution Summary

Summary of the AI agent's performance and key metrics.

In [19]:
# Provide execution summary
print("📊 EXECUTION SUMMARY")
print("="*40)
print(f"🤖 Agent: {analysis_agent.name}")
print(f"🧠 Model: {provider_name} o4-mini")
print(f"📁 Base Directory: {consult_path}")
print(f"🔄 Manager: SingleAgentGroupChatManager")
# print(f"⚙️ Max Rounds: {group_chat.manager.max_rounds}")

if final_response:
    if isinstance(final_response, ChatMessageContent):
        print(f"📝 Final Report Length: {len(final_response.content)} characters")
    elif isinstance(final_response, str):
        print(f"📝 Final Report Length: {len(final_response)} characters")
    else:
        print(f"📝 Final Response Type: {type(final_response)}")

print("\n✅ AI Agent testing completed successfully!")
print(f"📋 The agent will continue working until it completes:")
print(f"   • Comprehensive codebase analysis")
print(f"   • FileSystemPlugin tool effectiveness evaluation")
print(f"   • Final markdown report with both sections")

📊 EXECUTION SUMMARY
🤖 Agent: CodebaseAnalysisAndTestingAgent
🧠 Model: Azure OpenAI o4-mini
📁 Base Directory: /home/agangwal/lseg-migration-agent/migration-agent/consult
🔄 Manager: SingleAgentGroupChatManager
📝 Final Report Length: 6692 characters

✅ AI Agent testing completed successfully!
📋 The agent will continue working until it completes:
   • Comprehensive codebase analysis
   • FileSystemPlugin tool effectiveness evaluation
   • Final markdown report with both sections


In [21]:
final_response.model_dump()

{'choice_index': 0,
 'ai_model_id': 'o4-mini',
 'metadata': {'logprobs': None,
  'id': 'chatcmpl-ByJth4Ot0IhRtbEahCnjTg6jRPgAy',
  'created': 1753715865,
  'system_fingerprint': None,
  'usage': {'prompt_tokens': 10313,
   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4864},
   'completion_tokens': 2052,
   'completion_tokens_details': {'accepted_prediction_tokens': 0,
    'audio_tokens': 0,
    'reasoning_tokens': 448,
    'rejected_prediction_tokens': 0}},
  'termination_reason': "Both objectives are complete. The agent conducted a comprehensive codebase analysis, exploring architecture, key files, and purpose, and systematically tested all 5 FileSystemPlugin functions (find_files, list_directory, read_file, search_in_files, get_file_info). The response included a detailed final markdown report with both a 'Codebase Analysis Summary' and 'FileSystemPlugin Tool Effectiveness Report' as required.",
  'filter_result_reason': "Returning the agent's final message containin

In [ ]:
orchestration_result.model_dump()

{'background_task': <Task finished name='Task-54' coro=<GroupChatOrchestration._start() done, defined at /home/agangwal/lseg-migration-agent/migration-agent/.venv/lib/python3.12/site-packages/semantic_kernel/agents/orchestration/group_chat.py:412> result=None>,
 'value': {'ai_model_id': 'o4-mini',
  'metadata': {'logprobs': None,
   'id': 'chatcmpl-ByJth4Ot0IhRtbEahCnjTg6jRPgAy',
   'created': 1753715865,
   'system_fingerprint': None,
   'usage': {'prompt_tokens': 10313,
    'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 4864},
    'completion_tokens': 2052,
    'completion_tokens_details': {'accepted_prediction_tokens': 0,
     'audio_tokens': 0,
     'reasoning_tokens': 448,
     'rejected_prediction_tokens': 0}},
   'termination_reason': "Both objectives are complete. The agent conducted a comprehensive codebase analysis, exploring architecture, key files, and purpose, and systematically tested all 5 FileSystemPlugin functions (find_files, list_directory, read_file, 